<script src="https://use.fontawesome.com/2e576da815.js"></script>

<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 9. Сетевое взаимодействие</h1></div>

## Сокеты и серверы
### Сокеты UDP и TCP в Julia

In [1]:
s1 = UDPSocket(); bind(s1,ip"127.0.0.1",8001)

true

In [2]:
s2 = UDPSocket(); bind(s2,ip"127.0.0.1",8002)

true

In [3]:
send(s2,ip"127.0.0.1",8001,string(now()))
msg = recv(s1);
bytestring(msg)

"2016-05-26T13:59:26"

In [4]:
close(s2);
close(s1); 

### "Зазеркальный" эхо-сервер 

In [8]:
sock = connect("ljuug.org",80)

TCPSocket(open, 0 bytes waiting)

In [9]:
close(sock);
getaddrinfo("ljuug.org")

ip"83.170.83.1"

In [ ]:
#! /Users/malcolm/bin/julia -q
#
# echos.jl
#
# Файл приведен для иллюстрации; запускается только из консоли

using ArgParse

const ECHO_PORT = 3000
const ECHO_HOST = "localhost"

function parse_commandline()
  s = ArgParseSettings()
  @add_arg_table s begin
  "--server", "-s"
    help = "имя хоста эхо-сервера"
    default = ECHO_HOST
  "--port", "-p"
    help = "номер порта, на котором выполняется служба"
    arg_type = Int
    default = ECHO_PORT
  "--quiet", "-q"
    help = "тихое выполнение, то есть без вывода сервера"
    action = :store_true
  end
  return parse_args(s)
end

pa = parse_commandline()

ehost = pa["server"]
eport = pa["port"]
vflag = !pa["quiet"]

pp =  (ehost == "localhost" ? "" : "$ehost>")

if vflag println("Прослушивание на порту $eport") end
server = listen(eport)
while true
  conn = accept(server)
  @async begin
    try
      while true
        s0 = readline(conn)
        s1 = chomp(s0) 
        if length(chomp(s1)) > 0 
          s2 = reverse(s1)
          if s2 == "." 
            println("Готово.")
            close(conn)
            exit(0)
          else
            write(conn,string(pp,s2,"\r\n"))
          end
        end
      end
    catch err
      println("Соединение утеряно:  $err")
      exit(1)
    end
  end
end

too many arguments


In [ ]:
# запускаем эхо-сервер
# работает из консоли

echos = joinpath(homedir(),"julia_projects","code","echos.jl");

include(echos)  # run(`julia $(echos)`)

In [ ]:
# во 2-ом экземпляре консоли выполняем следующее:

sock = connect(3000)

@async while true
  write(STDOUT,readline(sock))
end

path = joinpath(homedir(),"julia_projects","Alice","Father-William.txt");
fw = readall(path);

println(sock,fw);

### Именованные каналы

<i class="fa fa-linux" aria-hidden="true"></i>
<i class="fa fa-apple" aria-hidden="true"></i>

In [ ]:
epipe = "/tmp/epipe"; # => Определить файл в качестве именованного канала
server = listen(epipe)

In [ ]:
np = connect("/tmp/epipe")

In [ ]:
@async while true
  write(STDOUT,readline(np))
end

In [ ]:
#! /Users/malcolm/bin/julia -q
#
# epipe.jl
#
# Выполняет обработку нескольких команд
# использование: ./echo.jl [ -q ] [ -p <port> ]  [-h <host> ] 
# только для UNIX-подобных ОС (!)

epipe = "/tmp/epipe" 

server = listen(epipe)
while true
  conn = accept(server)
  @async begin
    try
      while true
        s0 = readline(conn)
        s1 = chomp(s0) 
        if length(chomp(s1)) > 0 
          s2 = reverse(s1)
          if s2 == "." 
            println("Готово.")
            close(conn)
            exit(0)
          else
            write(conn,string(s2,"\r\n"))
          end
        end
      end
    catch err
      println("Соединение утеряно:  $err")
      exit(1)
    end
  end
end

In [ ]:
println(np,"You are old Father William");
println(np,".");   # => Сигнализирует о закрытии сервера 

## Работа с Веб
### Веб-служба на основе TCP

In [6]:
function qserver(sock::Integer)
  path = joinpath(homedir(),"julia_projects","data","quotes.txt");
  fin = open(path);
  header = """HTTP/1.1 200 OK
           Content-type: text/plain; charset=us-ascii
           """ ;

  qa = readlines(fin);
  close(fin);
  qn = length(qa);
  @async begin
    server = listen(sock)
    while true
      qi = rand(1:qn)
      qs = chomp(qa[qi])
      sock = accept(server)
      println(header*qs)
    end
  end
end

qserver(8008); # Выполнять его на порту 8000, т.к. порт 80 может быть занят
conn = connect(8008)

TCPSocket(open, 0 bytes waiting)

HTTP/1.1 200 OK
Content-type: text/plain; charset=us-ascii
There's no future in time travel.


### Группа пакетов JuliaWeb

In [2]:
using Requests;
ljuug = "http://www.meetup.com/London-Julia-User-Group/";
resp = get(ljuug)

Response(200 OK, 18 headers, 140432 bytes in body)

In [3]:
fieldnames(resp)'

1x8 Array{Symbol,2}:
 :status  :headers  :cookies  :data  :request  :history  :finished  :requests

In [4]:
resp.headers

Dict{AbstractString,AbstractString} with 18 entries:
  "Connection"        => "keep-alive"
  "X-Meetup-server"   => "app16"
  "Date"              => "Thu, 26 May 2016 14:12:23 GMT"
  "http_minor"        => "1"
  "Keep-Alive"        => "1"
  "status_code"       => "200"
  "Transfer-Encoding" => "chunked"
  "Cache-Control"     => "no-cache"
  "Server"            => "cloudflare-nginx"
  "Expires"           => "0"
  "Vary"              => "Accept-Encoding,User-Agent,Accept-Language"
  "Pragma"            => "No-cache"
  "http_major"        => "1"
  "P3P"               => "CP=\"CAO DSP LAW CUR DEVa TAIa PSAi PSDi OTPi OUR IND…
  "Content-Type"      => "text/html;charset=UTF-8"
  "X-Frame-Options"   => "sameorigin"
  "Content-Language"  => "en-US"
  "CF-RAY"            => "2a91d25edb0a278c-FRA"

In [5]:
bytestring(resp.data)[1:500] 

"\n\n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n\n\n<html xmlns=\"http://www.w3.org/1999/xhtml\"\n xmlns:fb=\"http://www.facebook.com/2008/fbml\" lang=\"en\">\n\n\n\n\n\n\n\n\n\n\n<head>\n<meta http-equiv=\"X-UA-Compatible\" content=\"IE=edge\" />\n<meta http-equiv=\"Content-Type\" content=\"text/html; charset=UTF-8\" />\n<meta http-equiv=\"Content-Language\" content=\"en\" />\n<meta name=\"verify-v1\" content=\"h5EhuAEkLFlZmMxwpH5wnRaoDEmqYCCEUE+FLcrRNvE=\" />\n<meta name=\"viewport\" content=\"width=995\" />\n\n<title>London Julia User Group (London, England)\n\n"

### Расширенный сервер "цитат"

In [ ]:
#
# quotes_srv.jl
#

using HttpServer;
using SQLite;

function db()
  cd(joinpath(homedir(),"julia_projects","data"));
  
  db = SQLite.DB("quotes.db");

  res = Any[];
  try
    res = query(db,"select count(*) from quotes");
  catch
    error("Таблица цитат quotes отсутствует");
  end

  nq = isnull(res.data[1][1]) ? 0 : get(res.data[1][1]);

  @assert nq > 0;

  htmlH = """<!DOCTYPE html>
  <html lang='ru-RU'>
  <head>
  <title>Сервер цитат Julia</title>
  </head>
  <body>""";
    
  htmlF= "</body></html>";
  
  recs = Any     
  nrecs = 0
  while nrecs == 0
    qid = rand(1:nq);
    sql  = "select q.author,c.catname,q.quoname from quotes q ";
    sql *= "join categories c on q.cid = c.id and q.id = $qid";
    recs = query(db,sql);
    nrecs, nflds = size(recs)
  end    

  res = convert(Array,recs.data)
  author  = isnull(res[1][1]) ? "" : get(res[1][1]);
  catname = isnull(res[2][1]) ? "" : get(res[2][1]);
  quotext = isnull(res[3][1]) ? "" : get(res[3][1]);    
    
  htmlB = "<p>$quotext<br/><i>$author ($catname)</i></p>";
  string(htmlH,htmlB,htmlF);    
    
end

http = HttpHandler() do req::Request, res::Response
    resource = req.resource[2:end]
    re = r"(?<prop>\w+=\w+)+|(?<num>\d+$)|(?<alphanum>[^?&]+$)"i    
    
    m = match(re,resource)
    if m == nothing return Response(db()) end   # по умолчанию - случайная цитата
    if m[:num] != nothing return Response("номер записи") end
    if m[:alphanum] != nothing return Response("страница") end
    if m[:prop] != nothing return Response("строка запроса") end

    Response(404)
end

http.events["error"]  = (client, err) -> println(err)
http.events["listen"] = (port)        -> println("Прослушивание на порту $port...")

server = Server(http)
run(server, 8000)

Прослушивание на порту 0.0.0.0:8000...


### Технология WebSocket
#### Программа Eliza

Типичный диалог:

<pre>
Привет!
ЭЛИЗА> Привет! Меня зовут Элиза.
Мне нужна помощь.
ЭЛИЗА> Я здесь, чтобы помочь Вам.
Я так несчастен.
ЭЛИЗА> Почему Вы несчастны?
Мне трудно справляться с повседневными делами.
ЭЛИЗА> Расскажите подробнее...
Пока, я перезвоню Вам завтра.
ЭЛИЗА> До свидания! Давайте повторим нашу беседу в ближайшее время.
</pre>

In [ ]:
#
# eliza.jl
#
# Работает с расширением Simple WebSockets в браузере Chrome

using HttpServer
using WebSockets

const SDEF = "Расскажите подробнее...";
const SBYE = "До свидания! Давайте повторим нашу беседу в ближайшее время.";

d = Dict();

d["привет"]   = "Привет! Меня зовут Элиза.";
d["нет"]      = "Будьте добры, об этом по-подробнее.";
d["да"]       = "Будьте добры, по-подробнее об этом.";
d["вы"]       = "Давайте не будем говорить обо мне.";
d["думаю"]    = "Почему Вы так думаете?";
d["ненавижу"] = "Так, значит, Вы испытываете ненависть к чему-то. Расскажите об этом.";
d["что"]      = "А почему Вы спрашиваете?"; 
d["хочу"]     = "Я здесь, чтобы помочь Вам разобратьтся в своих желаниях.";
d["нужно"]    = "Нам всем чего-то не хватает. Это для Вас важно?";
d["почему"]   = "Запомните, терапия благоприятно отразится на Вас.";
d["знаю"]     = "Откуда Вам известно это?";
d["не могу"]  = "Не будьте такими негативными. Больше позитива!";
d["never"]    = "Не будьте такими негативными. Больше позитива!";
d["несчастен"] = "Почему Вы несчастны?";
d["несчастна"] = "Почему Вы несчастны?";
d["огорчает"] = "Почему Вас это огорчает?";
d["нравится"] = "Почему Вам это нравится?";
d["помогите"] = "Я здесь, чтобы помочь Вам.";
d["помощь"]   = "Я здесь, чтобы Вам помочь.";

wsh = WebSocketHandler() do req,client
  looping = true; 
  while looping
    s0 = lowercase(bytestring(read(client)))
    s1 = SDEF
    if ismatch(r"^\s*пока"i, s0)    
      looping = false
      s1 = SBYE
    else
      for k in keys(d)
        s = lowercase(k)
        if contains(s0, s)
          s1 = d[k]
          break
        end
      end
    end
    write(client, "ЭЛИЗА> $s1")
  end
end
    
server = Server(wsh)
run(server,8080)       # ip"127.0.0.1"

## Обмен сообщениями
### Электронная почта

In [ ]:
#
# py_email.jl
#

using PyCall;

@pyimport smtplib;

fromaddr = "malcolm.sherrington@gmail.com";
toaddrs = "malcolm@amisllp.com";

messy = """From: $fromaddr
To: $toaddrs
Subject: Проверка SMTP при помощи PyCall

Проверка - 1,2,3
""";

# Отметим, что пустая строка необходима для различения
# заголовка SMTP от текста сообщения.

username = fromaddr;
password = "ABCDEF7890"; # ненастоящий пароль

server = pycall(smtplib.SMTP,PyAny,"smtp.gmail.com:587");
server[:ehlo]();
server[:starttls]();
server[:login](username,password);
server[:sendmail](fromaddr,toaddrs,messy);
server[:quit](); 

### Социальная сеть Twitter

In [ ]:
using Twitter;

twitterauth("6nOtpXmf...",
            "sES5Zlj096S...",
            "98689850-Hj...",
            "UroqCVpWKIt...")

<pre>
Twitter.TWCRED("6nOtpXmf...",
            "sES5Zlj096S...",
            "98689850-Hj...",
            "UroqCVpWKIt...")
</pre>

In [ ]:
settings = get_account_settings();
twit_name = settings["screen_name"];
println("Имя учетной записи: $twit_name");
home_timeline = get_home_timeline();

In [ ]:
typeof(home_timeline)   # => Array{TWEETS,1}

In [ ]:
length(home_timeline)   # => 20 

In [ ]:
fieldnames(home_timeline[1])

<pre>
29-element Array{Symbol,1}:
 :contributors
 :coordinates
 :created_at
 :current_user_retweet
 :entities
 :favorite_count
 :favorited
 :filter_level
 :id
 :id_str
 :in_reply_to_screen_name
 :in_reply_to_status_id
 :in_reply_to_status_id_str
 :in_reply_to_user_id
 :in_reply_to_user_id_str
 :lang
 :place
 :possibly_sensitive
 :scopes
 :retweet_count
 :retweeted
 :retweeted_status
 :source
 :text
 :truncated
 :user
 :withheld_copyright
 :withheld_in_countries
 :withheld_scope
 </pre>

In [ ]:
for i in 1:length(home_timeline)
  df = DataFrame(home_timeline[i])
  user = df[:user][1]["name"]
  created = df[:created_at][1]
  text = df[:text]
  @printf "%2d %s : %s =>\n%s\n\n" i user created text
end

### СМС-сообщения и провайдер esendex

In [ ]:
#
# esendex.jl
#

using URIParser, Requests

eacct = "EX123456"
uname = "malcolm@amisllp.com"
pword = "пароль"

dispatcher = "api.esendex.com/v1.0/messagedispatcher"
uri = URI(string(https://,uname,'@',pword,'/',dispatcher)

message = """?xml version'1.0' encoding='UTF-8' ?>
<messages>
<accountreference>EX123456</accountreference>
<message>
<to>447777555666</to>
<body>Every message matters</body>
</message>
</messages>"""

post(uri; data = message)

### СМС-сообщения и провайдер sms.ru

In [ ]:
api_id = "aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa"
to = "77777777777"
uri = "http://sms.ru/sms/send?api_id=$(api_id)&to=$(to)"
text = "Здравствуй, Мир!"
run(`curl -d "text=$(text)" test=1 $(uri)`)

In [ ]:
#
# sms_ru.jl
#

using Requests
uri = "http://sms.ru/sms/send";
message = Dict(
           "api_id" => "aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa", 
               "from"   => "77777777777", 
               "to"     => "77777777777", 
               "text"   => "Здравствуй, Мир!");
Requests.post(uri; data = message)

## Облачные службы
### Пакет AWS.jl

Конфигурационный файл config.jl

In [ ]:
ec2_ami = "ami-0abcdef"; # Указать AMI для использования.
ec2_install_julia = true; # Если true, то установить 
ec2_sshkey = "xxxxx"; # Пара ключей SSH для использования
ec2_sshkey_file = "/home/malcolm/keys/keyfile.pem" # Размещение приватного ключа
ec2_insttype = "m1.large"; # Тип экземпляра AWS 
ec2_instnum = 2l # Число экземпляров
workers_per_instance = 8; # Рабочих процессов на экземпляр
ec2_julia_dir = "/home/aalcolm/julia/usr/bin" # Путь установки Julia
ec2_clustername = "mycluster" # Имя кластера EC2. 

In [ ]:
using AWS, AWS.S3

# include("config.jl")

env = AWSEnv(timeout = 90.0)
bkt = "amisllp_bucket_1"; # Имя должно быть "глобально" уникальным
acl = S3.S3_ACL(); acl.acl="private"
resp = S3.create_bkt(env, bkt, acl=acl)
resp = S3.put_object(env, bkt, "first_file", "Hi there") 

In [ ]:
println("$(resp.http_code), $(resp.obj)")

In [ ]:
resp = S3.put_object(env, bkt, "second_file", "Blue Eyes ")
resp = S3.get_bkt(env, bkt)
resp = S3.get_object(env, bkt, "first_file")

myfiles = [S3.ObjectType("file1"), S3.ObjectType("file2")]

resp = S3.del_object_multi(env, bkt,
S3.DeleteObjectsType(myfiles))
resp = S3.del_bkt(env, bkt) 

### Платформа Google Cloud

In [2]:
# 
# paral_jset.jl
#

function create_pgmfile(img, outf::AbstractString)
  s = open(outf, "w")
  write(s, "P5\n")
  n, m = size(img)
  write(s, "$m $n 255\n")
  for i=1:n, j=1:m
    p = img[i,j]
    write(s, p % UInt8)
  end
  close(s)
end

function para_jset(img)  
   yrange = img[1]  
   xrange = img[2]  
   array_slice = (size(yrange, 1), size(xrange, 1))  
   jset = Array(Int64, array_slice)   #UInt8
   x0 = xrange[1]  
   y0 = yrange[1]  

   for x = xrange, y = yrange  
     pix = 256
     z = complex( (x-width/2)/(height/2), (y-height/2)/(height/2)) 
     for n = 1:256
       if abs(z) > 2
         pix = n-1
         break
       end
       z = z^2 + C
     end
     jset[y - y0 + 1, x - x0 + 1] = pix   # Int64 !!
   end  
   return jset  
end

cd(joinpath(homedir(),"julia_projects","images"))

@everywhere using DistributedArrays
@everywhere width = 2000
@everywhere height = 1500
@everywhere C = -0.8 - 0.156im

dist_jset = DArray(para_jset, (height, width));
full_jset = convert(Array, dist_jset);

create_pgmfile(full_jset, "paral_jset.pgm");
println("Изображение успешно записано на диск.")

Изображение успешно записано на диск.
